In [24]:
import pandas as pd
import networkx as nx
from itertools import combinations
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
import numpy as np

In [25]:
color_combinations = pd.read_csv('combinaciones_colores.csv')

In [26]:
df = pd.read_csv('Datos/transformados/df_resultado.csv')
df.pop('Unnamed: 0')
df.head()

,id_variant,id,title,season,hexadecimal,family_id,id_hex,weather,nivel,formalidad,...,foulard_type,size,skirt_type,message,hood,back_neckline,shoulders_pad,bag_type,hips_volume,chest_volume
0,fbc2f719-e038-4243-892c-7bd2ca6e80b5,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,Onljade Cardigan knit,8,000000,141f32d6-a5af-49bf-b019-fd28354977f1,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf_000000,cold,3.0,Formal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c85666c0-a8ba-42f4-a30f-17687ad51ff5,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,Onljade Cardigan knit,8,DFC8B2,141f32d6-a5af-49bf-b019-fd28354977f1,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf_DFC8B2,cold,3.0,Formal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,93db9797-3a66-4889-be9c-5d3522da96a1,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,Onljade Cardigan knit,8,B94600,141f32d6-a5af-49bf-b019-fd28354977f1,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf_B94600,cold,3.0,Formal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0c1010aa-2f25-4c55-9f95-b9bc133f3307,6e6b4255-006a-4d13-b312-3b1426290d8e,Poly Pant deeluxe,7,B11730,6598b655-3e19-448c-8b4f-ff6a29447b34,6e6b4255-006a-4d13-b312-3b1426290d8e_B11730,warm,1.1,Formal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215a323c-9721-4879-9aa6-5cf30b5b21f0,6549c1b1-989e-432a-bdc9-b603c438b310,Onleden Sweater knt,9,FFFF00,1b93b30c-833f-45a8-ba69-c21d2e7f4824,6549c1b1-989e-432a-bdc9-b603c438b310_FFFF00,warm,3.0,Informal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df_s7 = df[df['season']==7]
df_s8 = df[df['season']==8]
df_s9 = df[df['season']==9]

**GRAFO**

- Eliminar columnas : top_type, finishing, closing, neck, detail, sleeve, sleeve_long, sizing, composition,  fabric, hip_contour, waist_contour, chest_contour, sleeve_long_cm, long_cm, gripper, basic, down_part_type, bottom, type_of_length, neckline, outside_type, cover, light, alerts, cut, rubber_waist, label, dress_type, foulard_type, size, skirt_type, message, hood, back_neckline, shoulders_pad, bag_type, hips_volume, chest_volume, shot, thickness

In [27]:
columnas_a_eliminar = [
    'top_type', 'finishing', 'closing', 'neck', 'detail', 'sleeve', 'sleeve_long', 'sizing', 
    'composition', 'fabric', 'hip_contour', 'waist_contour', 'chest_contour', 'sleeve_long_cm', 
    'long_cm', 'gripper', 'basic', 'down_part_type', 'bottom', 'type_of_length', 'neckline', 
    'outside_type', 'cover', 'light', 'alerts', 'cut', 'rubber_waist', 'label', 'dress_type', 
    'foulard_type', 'size', 'skirt_type', 'message', 'hood', 'back_neckline', 'shoulders_pad', 
    'bag_type', 'hips_volume', 'chest_volume', 'shot', 'thicknees', 'weather.1', 'id_hex'
]

df_limpio = df.drop(columns=columnas_a_eliminar)


In [28]:
df_limpio.head()

,id_variant,id,title,season,hexadecimal,family_id,weather,nivel,formalidad,adventurous,print,style,fit,application
0,fbc2f719-e038-4243-892c-7bd2ca6e80b5,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,Onljade Cardigan knit,8,000000,141f32d6-a5af-49bf-b019-fd28354977f1,cold,3.0,Formal,1,smooth,classic,straight,freetime
1,c85666c0-a8ba-42f4-a30f-17687ad51ff5,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,Onljade Cardigan knit,8,DFC8B2,141f32d6-a5af-49bf-b019-fd28354977f1,cold,3.0,Formal,1,smooth,classic,straight,freetime
2,93db9797-3a66-4889-be9c-5d3522da96a1,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,Onljade Cardigan knit,8,B94600,141f32d6-a5af-49bf-b019-fd28354977f1,cold,3.0,Formal,1,smooth,classic,straight,freetime
3,0c1010aa-2f25-4c55-9f95-b9bc133f3307,6e6b4255-006a-4d13-b312-3b1426290d8e,Poly Pant deeluxe,7,B11730,6598b655-3e19-448c-8b4f-ff6a29447b34,warm,1.1,Formal,1,smooth,street,tight,freetime
4,215a323c-9721-4879-9aa6-5cf30b5b21f0,6549c1b1-989e-432a-bdc9-b603c438b310,Onleden Sweater knt,9,FFFF00,1b93b30c-833f-45a8-ba69-c21d2e7f4824,warm,3.0,Informal,1,smooth,boho,loose,freetime


In [29]:
df_limpio['id_nodo'] = range(len(df))

G = nx.Graph()

for _, row in df_limpio.iterrows():
    G.add_node(row['id_nodo'], feature=[
        row['season'], row['hexadecimal'], row['weather'], 
        row['nivel'], row['formalidad'], row['adventurous'], row['print'], row['style'], 
        row['fit'], row['application']
    ], label=row['id_nodo'])


for node in G.nodes(data=True):
    print(node)

(0, {'feature': [8, '000000', 'cold', 3.0, 'Formal', 1, 'smooth', 'classic', 'straight', 'freetime'], 'label': 0})
(1, {'feature': [8, 'DFC8B2', 'cold', 3.0, 'Formal', 1, 'smooth', 'classic', 'straight', 'freetime'], 'label': 1})
(2, {'feature': [8, 'B94600', 'cold', 3.0, 'Formal', 1, 'smooth', 'classic', 'straight', 'freetime'], 'label': 2})
(3, {'feature': [7, 'B11730', 'warm', 1.1, 'Formal', 1, 'smooth', 'street', 'tight', 'freetime'], 'label': 3})
(4, {'feature': [9, 'FFFF00', 'warm', 3.0, 'Informal', 1, 'smooth', 'boho', 'loose', 'freetime'], 'label': 4})
(5, {'feature': [9, 'ECE9D6', 'warm', 3.0, 'Informal', 1, 'smooth', 'boho', 'loose', 'freetime'], 'label': 5})
(6, {'feature': [9, '00008b', 'warm', 3.0, 'Informal', 1, 'smooth', 'boho', 'loose', 'freetime'], 'label': 6})
(7, {'feature': [8, '00008b', 'warm', 3.1, 'Informal', 2, 'floral', 'street', 'loose', 'freetime'], 'label': 7})
(8, {'feature': [8, '000000', 'warm', 3.1, 'Informal', 2, 'floral', 'street', 'loose', 'freetime']

In [30]:
# Comprobar que todos los nodos tienen el mismo número de features
feature_length = None
all_equal = True

for node, data in G.nodes(data=True):
    current_length = len(data['feature'])
    
    if feature_length is None:
        feature_length = current_length
    elif feature_length != current_length:
        all_equal = False
        break

if all_equal:
    print(f"Todos los nodos tienen el mismo número de features: {feature_length}")
else:
    print("No todos los nodos tienen el mismo número de features.")


Todos los nodos tienen el mismo número de features: 10


___
## **SEASON7**
___

In [35]:
df_limpio_s7 = df_limpio[df_limpio['season']==7]

df_limpio_s7['id_nodo'] = range(len(df_limpio_s7))

G7 = nx.Graph()

for _, row in df_limpio_s7.iterrows():
    G7.add_node(row['id_nodo'], feature=[
        row['season'], row['hexadecimal'], row['weather'], 
        row['nivel'], row['formalidad'], row['adventurous'], row['print'], row['style'], 
        row['fit'], row['application']
    ], label=row['id_nodo'])


for node in G7.nodes(data=True):
    print(node)

(0, {'feature': [7, 'B11730', 'warm', 1.1, 'Formal', 1, 'smooth', 'street', 'tight', 'freetime'], 'label': 0})
(1, {'feature': [7, 'ECE9D6', 'warm', 3.0, 'Informal', 2, 'horizontal_stripes', 'classic', 'straight', 'work'], 'label': 1})
(2, {'feature': [7, '5F5E5E', 'warm', 2.0, 'Informal', 1, 'smooth', 'classic', 'straight', 'freetime'], 'label': 2})
(3, {'feature': [7, '00008b', 'warm', 2.0, 'Informal', 1, 'horizontal_stripes', 'casual', 'straight', 'freetime'], 'label': 3})
(4, {'feature': [7, '000000', 'cold', 3.1, 'Informal', 3, 'sheets', 'casual', 'loose', 'freetime'], 'label': 4})
(5, {'feature': [7, '00008b', 'cold', 3.1, 'Informal', 2, 'smooth', 'street', 'tight', 'freetime'], 'label': 5})
(6, {'feature': [7, '666633', 'cold', 3.1, 'Informal', 3, 'smooth', 'street', 'straight', 'freetime'], 'label': 6})
(7, {'feature': [7, '00008b', 'cold', 3.1, 'Informal', 3, 'smooth', 'street', 'straight', 'freetime'], 'label': 7})
(8, {'feature': [7, '008000', 'warm', 2.0, 'Informal', 1, 've

C:\Users\usuario\AppData\Local\Temp\ipykernel_5224\1522886681.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio_s7['id_nodo'] = range(len(df_limpio_s7))


**ARISTAS**

Si una prenda es de nivel 1.2, no puede combinarse con una prenda de nivel 2.
En los demás casos, las prendas no pueden combinarse si tienen niveles principales iguales, excepto si ambos son del nivel 3.
Separación de la lógica para el caso especial de 1.2: Permitir que las prendas de nivel 3.x se combinen con otras de nivel 3.x siempre que tengan subniveles diferentes.

## **DE MOMMENTO SE USA MUESTRA REDUCIDA PARA VER QUE FUNCIONA BIEN**

In [40]:
def generar_aristas_optimizado(grafo, color_comb_df):
    # Extraer nodos y características
    nodes_data = pd.DataFrame(
        [(node, data['feature']) for node, data in grafo.nodes(data=True)],
        columns=["node", "feature"]
    )
    
    # Convertir características a columnas separadas
    features_df = pd.DataFrame(nodes_data['feature'].tolist(), columns=[ 
        "temporada", "color", "temperatura", "nivel_prenda", 
        "tipo", "adventurous", "textura", "estilo", "corte", "uso"
    ])
    nodes_data = pd.concat([nodes_data, features_df], axis=1)

    # Convertir adventurous a valores numéricos
    nodes_data['adventurous'] = pd.to_numeric(nodes_data['adventurous'], errors='coerce')

    sample_nodes = nodes_data.sample(n=20, random_state=42)  # Selecciona 10 nodos al azar

    # Crear una matriz de combinaciones de nodos
    idx_combinations = np.triu_indices(len(sample_nodes), k=1)
    node1_indices = idx_combinations[0]
    node2_indices = idx_combinations[1]

    # Verificación de reglas usando lambda
    verificar_nivel = lambda row1, row2: (
        (row1['nivel_prenda'] == 1.2 and row2['nivel_prenda'] != 2) or 
        (row2['nivel_prenda'] == 1.2 and row1['nivel_prenda'] != 2) or 
        (row1['nivel_prenda'] == 3 and row2['nivel_prenda'] == 3 and row1['nivel_prenda'] != row2['nivel_prenda']) or 
        (row1['nivel_prenda'] != row2['nivel_prenda'] and not (
            (row1['nivel_prenda'] == 1.2 and row2['nivel_prenda'] == 2) or 
            (row2['nivel_prenda'] == 1.2 and row1['nivel_prenda'] == 2)
        ))
    )

    valid_color_combinations = dict(zip(zip(color_comb_df['color1'], color_comb_df['color2']), color_comb_df['combination']))

    # Otras verificaciones (temporalidad, color, etc.)
    verificar_otros = lambda row1, row2: all([
        valid_color_combinations.get((row1['color'], row2['color']), 0) == 1,
        row1['temporada'] == row2['temporada'],
        row1['color'] != row2['color'],
        row1['temperatura'] == row2['temperatura'],
        abs(row1['adventurous'] - row2['adventurous']) <= 1
    ])

    # Generar las aristas
    valid_edges = []
    for idx1, idx2 in zip(node1_indices, node2_indices):
        row1, row2 = sample_nodes.iloc[idx1], sample_nodes.iloc[idx2]
        
        # Imprimir los nodos y sus características
        print(f"Probando nodos {row1['node']} y {row2['node']}:")
        print(f"  Nodo 1 - {row1['node']}. Características: {row1[['temporada', 'color', 'temperatura', 'nivel_prenda', 'adventurous']].to_dict()}")
        print(f"  Nodo 2 - {row2['node']}. Características: {row2[['temporada', 'color', 'temperatura', 'nivel_prenda', 'adventurous']].to_dict()}")
        
        # Primero, verificar el color
        if valid_color_combinations.get((row1['color'], row2['color']), 0) != 1:
            print(f"  No cumple la regla de combinación de colores ({row1['color']}, {row2['color']}). No se generará la arista.")
            continue
        
        # Luego, verificar el nivel de prenda
        if not verificar_nivel(row1, row2):
            print(f"  No cumple la regla 'nivel_prenda'. No se generará la arista.")
            continue
        
        # Verificación detallada de la regla 'otros' si se cumplen las anteriores
        if not verificar_otros(row1, row2):
            print(f"  No cumple la regla 'otros'. No se generará la arista por las siguientes razones:")
            if row1['temporada'] != row2['temporada']:
                print(f"    - La temporada no coincide: {row1['temporada']} != {row2['temporada']}.")
            if row1['color'] == row2['color']:
                print(f"    - Los colores son iguales.")
            if row1['temperatura'] != row2['temperatura']:
                print(f"    - Las temperaturas no coinciden: {row1['temperatura']} != {row2['temperatura']}.")
            if abs(row1['adventurous'] - row2['adventurous']) > 1:
                print(f"    - La diferencia en adventurous es mayor a 1: {row1['adventurous']} - {row2['adventurous']}.")
            continue
        
        # Si ambas condiciones se cumplen, añadir la arista válida
        valid_edges.append((row1['node'], row2['node']))
        print(f"  ¡Combinación válida! Se generó la arista entre {row1['node']} y {row2['node']}")
    
    return valid_edges


In [41]:
# Generar aristas
aristas = generar_aristas_optimizado(G, color_combinations)

# Agregar aristas al grafo
G7.add_edges_from(aristas)

print(f"Número de aristas generadas: {len(aristas)}")


Probando nodos 1259 y 1623:
  Nodo 1 - 1259. Características: {'temporada': 7, 'color': '87CEFA', 'temperatura': 'cold', 'nivel_prenda': 3.0, 'adventurous': 3}
  Nodo 2 - 1623. Características: {'temporada': 7, 'color': 'B94600', 'temperatura': 'warm', 'nivel_prenda': 2.0, 'adventurous': 2}
  No cumple la regla de combinación de colores (87CEFA, B94600). No se generará la arista.
Probando nodos 1259 y 611:
  Nodo 1 - 1259. Características: {'temporada': 7, 'color': '87CEFA', 'temperatura': 'cold', 'nivel_prenda': 3.0, 'adventurous': 3}
  Nodo 2 - 611. Características: {'temporada': 7, 'color': 'FFFF00', 'temperatura': 'warm', 'nivel_prenda': 2.0, 'adventurous': 2}
  No cumple la regla de combinación de colores (87CEFA, FFFF00). No se generará la arista.
Probando nodos 1259 y 514:
  Nodo 1 - 1259. Características: {'temporada': 7, 'color': '87CEFA', 'temperatura': 'cold', 'nivel_prenda': 3.0, 'adventurous': 3}
  Nodo 2 - 514. Características: {'temporada': 7, 'color': '000000', 'tempera

**Análisis del grafo**

In [ ]:
# Análisis del grafo
def analizar_grafo(G):
    analysis = {}
    
    # Número de nodos y aristas
    analysis['nodos'] = G.number_of_nodes()
    analysis['aristas'] = G.number_of_edges()
    
    # Densidad del grafo
    analysis['densidad'] = nx.density(G)
    
    # Componentes conectadas
    componentes = list(nx.connected_components(G))
    analysis['componentes_conectadas'] = len(componentes)
    analysis['tamaño_componentes'] = [len(c) for c in componentes]
    
    # Tamaño de la componente gigante
    analysis['tamaño_componente_gigante'] = max(analysis['tamaño_componentes']) if componentes else 0
    
    # Diámetro (solo si el grafo es conexo)
    if nx.is_connected(G):
        analysis['diametro'] = nx.diameter(G)
    else:
        analysis['diametro'] = "No aplica (grafo no conexo)"
    
    # Coeficiente de agrupamiento promedio
    analysis['coeficiente_agrupamiento'] = nx.average_clustering(G)
    
    # Grado promedio
    grados = dict(G.degree())
    analysis['grado_promedio'] = sum(grados.values()) / analysis['nodos'] if analysis['nodos'] > 0 else 0
    
    # Distribución del grado
    analysis['distribución_grado'] = {k: list(grados.values()).count(k) for k in set(grados.values())}
    
    # Centralidad de grado
    centralidad_grado = nx.degree_centrality(G)
    analysis['centralidad_grado_max'] = max(centralidad_grado.values())
    analysis['nodo_mas_central'] = max(centralidad_grado, key=centralidad_grado.get)
    
    # Centralidad de intermediación (betweenness centrality)
    centralidad_intermediacion = nx.betweenness_centrality(G)
    analysis['centralidad_intermediacion_max'] = max(centralidad_intermediacion.values())
    
    # Mostrar métricas
    for key, value in analysis.items():
        print(f"{key}: {value}")
    
    return analysis

# Aplicar el análisis
if __name__ == "__main__":
    resultado = analizar_grafo(G)


Nodos y Aristas: Número total de nodos y aristas.

Densidad del Grafo: Proporción entre el número de aristas y las posibles aristas máximas.

Componentes Conectadas: Número de componentes conectadas y el tamaño de cada una.

Tamaño de la Componente Gigante: Tamaño de la mayor componente conectada.

Diámetro: Distancia más larga entre dos nodos en la componente gigante (solo si es conexo).

Coeficiente de Agrupamiento Promedio: Mide qué tan conectados están los vecinos de cada nodo.

Grado Promedio: Promedio de conexiones por nodo.

Distribución del Grado: Frecuencia de cada grado (número de conexiones).

Centralidad de Grado: Nodo con más conexiones relativas en el grafo.

Centralidad de Intermediación: Nodo que conecta más flujos en el grafo.

In [ ]:
# Función para analizar componentes conectadas
def analizar_componentes_aisladas(G):
    # Obtener las componentes conectadas
    componentes = list(nx.connected_components(G))
    
    # Filtrar componentes de tamaño 1 (nodos aislados)
    nodos_aislados = [c for c in componentes if len(c) == 1]
    componentes_no_aisladas = [c for c in componentes if len(c) > 1]
    
    print(f"Total de componentes: {len(componentes)}")
    print(f"Componentes aisladas (tamaño 1): {len(nodos_aislados)}")
    print(f"Componentes no aisladas: {len(componentes_no_aisladas)}")
    print(f"Tamaños de las componentes no aisladas: {[len(c) for c in componentes_no_aisladas]}")
    
    return nodos_aislados, componentes_no_aisladas

# Aplicar el análisis
if __name__ == "__main__":
    nodos_aislados, componentes_no_aisladas = analizar_componentes_aisladas(G)


In [ ]:
# Nodos aislados
nodos_aislados_planos = [list(c)[0] for c in nodos_aislados]
print(f"Nodos aislados: {nodos_aislados_planos}")


In [ ]:
# Dibuja el grafo con componentes destacadas
plt.figure(figsize=(10, 10))
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=50)
nx.draw_networkx_edges(G, pos, alpha=0.3)
for c in componentes_no_aisladas:
    nx.draw_networkx_nodes(G.subgraph(c), pos, node_color='red', node_size=50)
for c in nodos_aislados:
    nx.draw_networkx_nodes(G.subgraph(c), pos, node_color='green', node_size=100)
plt.title("Componentes Aisladas y No Aisladas")
plt.show()


In [ ]:
# Función para graficar un grafo en 3D
def graficar_grafo_3d(G, sample_size=100):
    # Tomar una muestra aleatoria de nodos
    sampled_nodes = random.sample(G.nodes, min(sample_size, len(G.nodes)))
    G_sample = G.subgraph(sampled_nodes)
    
    # Generar posiciones 3D para los nodos
    pos = nx.spring_layout(G_sample, dim=3, seed=42)  # Usamos spring layout para distribuir los nodos
    
    # Crear el gráfico 3D
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Dibujar nodos
    for node, (x, y, z) in pos.items():
        ax.scatter(x, y, z, s=20, c='blue', alpha=0.7)
    
    # Dibujar aristas
    for edge in G_sample.edges:
        x_coords = [pos[edge[0]][0], pos[edge[1]][0]]
        y_coords = [pos[edge[0]][1], pos[edge[1]][1]]
        z_coords = [pos[edge[0]][2], pos[edge[1]][2]]
        ax.plot(x_coords, y_coords, z_coords, c='gray', alpha=0.5)
    
    # Ajustar la visualización
    ax.set_title("Visualización 3D del Grafo (Muestra de 100 nodos)")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    plt.show()

# Llamar a la función
if __name__ == "__main__":
    graficar_grafo_3d(G, sample_size=100)
